# load the data from excel

In [1]:
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# read the excel data
df = pd.read_excel('/app/notebooks/rapprochements/Operat/EFA.xlsx')

# print the df length
print('The length of the dataframe is: ', len(df))

The length of the dataframe is:  294047


In [2]:
# display df columns
print(df.columns)

Index(['id_EFA', 'etablissement_occupant_type',
       'etablissement - etablissement_id → identifiant',
       'etablissement - etablissement_id → nom', 'occupant_siren_id',
       'structure - occupant_siren_id → nom',
       'structure - occupant_siren_id → identifiant',
       'structure - occupant_siren_id → type',
       'denomination EFA si SIREN occupant', 'complement_nom_EFA',
       'adresse - adresse_id → commune_id', 'adresse - adresse_id → adresse',
       'adresse - adresse_id → complement_adresse',
       'adresse - adresse_id → latitude', 'adresse - adresse_id → longitude',
       'adresse - adresse_id → numero_insee',
       'commune - commune_id → code_postal', 'commune - commune_id → nom'],
      dtype='object')


# clean the addresses

In [8]:
df_for_geocoding = df[["id_EFA", "adresse - adresse_id → adresse", "commune - commune_id → code_postal", "commune - commune_id → nom", "adresse - adresse_id → numero_insee"]].copy()

# rename columns
df_for_geocoding.columns = ["id_EFA", "adresse", "code_postal", "commune", "cle_ban_initiale"]

# df_for_geocoding = df_for_geocoding.copy()

df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse"].str.replace(" BD ", " BOULEVARD ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" PL ", " PLACE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" AV ", " AVENUE ")

df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" ALL ", " ALLÉE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" IMP ", " IMPASSE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" SQ ", " SQUARE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" RTE ", " ROUTE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" CHE ", " CHEMIN ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" QU ", " QUAI ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace(" GAL DE GAULLE ", " GENERAL DE GAULLE ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace("CENTRE CIAL ", "CENTRE COMMERCIAL ")
df_for_geocoding["adresse_clean"] = df_for_geocoding["adresse_clean"].str.replace("C CIAL ", "CENTRE COMMERCIAL ")


df_for_geocoding.to_csv('/app/notebooks/rapprochements/Operat/EFA_for_geocoding.csv', index=False)

# geocode the addresses with BAN

In [9]:
from batid.services.geocoders import BanBatchGeocoder
from io import StringIO

geocoder = BanBatchGeocoder()
response = geocoder.geocode_file('/app/notebooks/rapprochements/Operat/EFA_for_geocoding.csv', columns=['adresse_clean', 'code_postal', 'commune'])
response_csv = StringIO(response.text)
df_geocoded = pd.read_csv(response_csv, sep=',')
df_geocoded.to_csv('/app/notebooks/rapprochements/Operat/EFA_geocoded.csv', index=False)

/tmp/ipykernel_1686/2859717419.py:7: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geocoded = pd.read_csv(response_csv, sep=',')


# join the data with OPERAT_RNB_Cas_EFA.xlsx

In [7]:
df_efa = pd.read_csv('/app/notebooks/rapprochements/Operat/EFA_geocoded.csv')
df_cas = pd.read_excel('/app/notebooks/rapprochements/Operat/OPERAT_RNB_Cas_EFA.xlsx')

# join the two dataframes with the id_EFA column
df = pd.merge(df_efa, df_cas, on='id_EFA', how='left')
print(df.head())

/tmp/ipykernel_19283/62312270.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_efa = pd.read_csv('/app/notebooks/rapprochements/Operat/EFA_geocoded.csv')


   id_EFA                       adresse code_postal     commune  \
0       5  RUE DE ST MICHEL A ANGOULEME     16000.0   ANGOULEME   
1       6         17 BD AUGUSTE BLANQUI     75013.0    PARIS 13   
2       8          LA CROIX DES ARCHERS     56200.0  LA GACILLY   
3       9          LA CROIX DES ARCHERS     56200.0  LA GACILLY   
4      11                       PL FOCH     20000.0     AJACCIO   

  cle_ban_initiale                 adresse_clean   latitude  longitude  \
0              NaN  RUE DE ST MICHEL A ANGOULEME  45.642992   0.114138   
1              NaN  17 BOULEVARD AUGUSTE BLANQUI  48.830380   2.353515   
2              NaN          LA CROIX DES ARCHERS  47.773274  -2.134821   
3              NaN          LA CROIX DES ARCHERS  47.773274  -2.134821   
4              NaN                       PL FOCH  41.918998   8.738901   

                                   result_label  result_score  \
0  Rue Saint Michel a Angouleme 16000 Angoulême      0.812457   
1      17 Boulevard Au

# find the corresponding buildings in the RNB

In [9]:
from batid.models import Building
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from concurrent.futures import ThreadPoolExecutor

# split the dataframe in 10 smaller dataframes
dfs = np.array_split(df, 10)

def get_buildings(row):
    ban_id = None
    # first choice, we pick the ban_id provided by Operat if value is not NaN
    if not pd.isnull(row['cle_ban_initiale']):
        ban_id = row['cle_ban_initiale']
    # second choice, we pick the ban_id provided by the geocoder
    elif row['result_score'] > 0.75 and row['result_type'] == 'housenumber':
        ban_id = row['result_id']
    
    if ban_id is None:
        return []
    else:
        buildings = Building.objects.filter(addresses_read_only__id=ban_id).values('rnb_id')
        return [b['rnb_id'] for b in buildings]

def execute(df):
    df_copy = df.copy()
    df_copy['rnb_ids'] = df_copy.apply(get_buildings, axis=1)
    return df_copy

with ThreadPoolExecutor(max_workers=10) as executor:
    results = executor.map(execute, dfs)
    df_results = pd.concat(results)
    df_results.to_csv('/app/notebooks/rapprochements/Operat/EFA_results.csv', index=False)

## prendre le cas EFA en compte pour l'interprétation des résultats

In [11]:
def coherence_cas_efa(row):
    cas_efa = row['cas_assujetissement']
    rnb_ids = row['rnb_ids']

    if cas_efa in ['1A', '1B', '2'] and len(rnb_ids) > 1:
        return 1
    elif cas_efa == '3' and len(rnb_ids) == 1:
        return 1
    else:
        return 0

df_results['incoherence_rnb_ids_avec_cas_efa'] = df_results.apply(coherence_cas_efa, axis=1)
df_results.to_csv('/app/notebooks/rapprochements/Operat/EFA_results.csv', index=False)


## quelques stats

In [15]:
# number of lines
print('Number of lines: ', len(df_results))
# count number of lines with rnb_ids
print('Number of lines with rnb_ids: ', len(df_results[df_results['rnb_ids'].apply(lambda x: len(x) > 0)]))
# count number of inconsistencies
print('Number of inconsistencies: ', len(df_results[df_results['incoherence_rnb_ids_avec_cas_efa'] == 1]))

# percentage of of lines with rnb_ids
print('Percentage of lines with rnb_ids: ', len(df_results[df_results['rnb_ids'].apply(lambda x: len(x) > 0)]) / len(df_results))

Number of lines:  294047
Number of lines with rnb_ids:  163762
Number of inconsistencies:  62976
Percentage of lines with rnb_ids:  0.5569245732825024


In [16]:
# samples of the results
df_sample = df_results.sample(30)
df_sample.to_csv('/app/notebooks/rapprochements/Operat/EFA_results_sample.csv', index=False)

## Export résultat final

In [18]:
df_results[['id_EFA', 'rnb_ids', 'cas_assujetissement']].to_csv('/app/notebooks/rapprochements/Operat/EFA_results_final.csv', index=False)